# Background
In just a paragraph or two, introduce your project. Example questions to consider: What is the current
situation with Lake Mead? What is the proposed change? Why does it matter? This section should include
citations in the text to at least two outside sources of information.

Potential sources (cite later/after use):
https://earth.org/lake-mead-and-colorado-river-basin-water-shortage/#:~:text=Formal%20Water%20Shortage%20Declared%20for,of%20natural%20and%20anthropogenic%20activities.

https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2007WR006704


The Colorado River, which originates in the Rocky mountains 

### Question
How does the cost-effectiveness of floatovoltaics vs shade balls compare?

Two proposed solutions to the Lake Mead water crisis are floatovoltaics - solar panels that would float on the surface of the lake - and shade balls - plastic spheres that block sunlight in order to reduce evaporation. 

The goal of this project is to compare the ratio of effectiveness at preventing evaporation to the cost of the proposed solution, which would allow us to determine which solution is worth putting time and money into in order to minimize the ongoing water crisis.

# Modeling

### Schematic


• Using a clear, diagrammatic way of documenting fluid flows, create a captioned figure illustrating
conservation of water mass for Lake Mead. Explain.

### Equations
• Write the system of equations, including at least one ODE, that will be used to solve for lake volume
and/or height as a function of time. Explain these equations in words, including why they are
appropriate for modeling the system of interest. Make sure at least one of the non-differential terms
in the primary ODE is not a constant1

[insert variable defenitions and values]




• Discuss the primary ODE for your system. What type of ODE is it? What type of behavior do you
expect from an ODE like this?
• State how you will solve the system of equations, including any initial values.


### Assumptions
When developing this model, we made a few key assumptions & acknowledged that certain factors are outside of our control:
1. The input rate of water from the Colorado River is constant
2. The output rate of water from Lake Mead is constant
3. The shape of the lake is a triangular prism

[justify each assumption]

# Validation
Using data and trends for Lake Mead, show that your baseline lake model (without floatovoltaics) is reasonable.
This will lend credibility to non-baseline predictions.

# Results & Discussion
Show, explain, and discuss the results of your model. Include, at the minimum, a plot of lake volume and/or
height over time with and without floatovoltaics. Make a recommendation based on your findings and
discuss the potential impacts of the recommended path. You may optionally choose to add conclusion section
summarizing findings.

The implementation of the project has three sections - the MVP and two iterations. 

### MVP 
The MVP assumes that the input and the output of the Colorado River are constant and contains basic math for the effect of having solar panels covering the lake.



In [ ]:
# MVP Plots

### Iteration 1 - Cost Effectiveness Analsysis
The first iteration models the input as the flow rate of the Colorado River and the output as the dam ODE. Additionally, the effect of using shade balls as opposed to solar panels covering the lake is included.

In [ ]:
# Iteration 1 plots

### Iteration 2 - Seasonal Changes
The second iteration of the model implements support for changing seasons in order to account for changes in water level at different points in time during the year.

In [ ]:
# Iteration 2 Plots

# References
